In [8]:
from haversine import haversine # 이동거리
# from geopy.geocoders import Nominatim # 역지오코딩(위도,경도->주소)
import requests
from monitor.models import Message
from monitor.geo import KakaoLocalAPI
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [5]:
# -------------------------------------------------------------------------------------------------
# 측정단말이 한곳에 머무는지 확인
#--------------------------------------------------------------------------------------------------
def call_staying_check(mdata):
    ''' 측정단말이 한곳에 머무는지 확인
        - 타사 측정단말에 문제가 발생하여 조치를 하거나 차량에 문제가 있거나 등 한곳에 오랫동안 멈는 경우가 있는데,
          이렇게 한곳에 멈춰 있는 경우 보고 대상임
        - 이동거리가 5미터 이내 연속해서 3회 이상 발생하면 한 곳에 머무는 것으로 판단 <- 기준확인 필요
        - return 'CALLSTAY'
    '''
    # 2022.02.22 - 처리대상 데이터를 속도측정 데이터('speed')에 한정하여 처리한다.  
    # mdata_list = mdata.phone.measurecalldata_set.all()
    mdata_list = mdata.phone.measurecalldata_set.filter(testNetworkType='speed').order_by("-currentCount")
    count = len(mdata_list)
    callstay = True
    # 이동거리를 확인하기 위해서는 측정값이 4건 이상 있어야 한다.
    if count >= 4:
        # for idx, md in enumerate(mdata_list[count-1::-1]):
        result = ''
        for idx, md in enumerate(mdata_list):
            if idx == 0:
                before_loc = (md.latitude, md.longitude)
            else:
                # 두 측정저점간의 이동거리를 계산한다. 
                current_loc = (md.latitude, md.longitude)
                distance = haversine(before_loc, current_loc) * 1000 # 미터(meters)
                # print("call_staying_check():", idx, str(md), distance, before_loc, current_loc)
                # 측정 단말기 이동거리가 5M 이상이 되면 한곳에 머무르지 않고, 이동하는 것으로 판단한다.
                result += str(before_loc)+ '/' + str(current_loc) + '/' + str(distance) + ','
                if distance > 5 :
                    callstay = False
                    break
                before_loc = current_loc
            if idx >= 3 : break
    else:
        callstay = False
    if callstay: 
        print(result)
        return 'CALLSTAY' 
    else:
        return None

In [6]:
mdata_list = MeasureCallData.objects.filter(testNetworkType='speed')
len(mdata_list)

184

In [7]:
for mdata in mdata_list:
    print("####")
    print(call_staying_check(mdata))

####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.89157479)/162.48676080484756,
None
####
(37.47297079, 126.89254859)/(37.47421097, 126.891

(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.33081059507235,
None
####
(37.49747459, 126.95732936)/(37.49714472, 126.95720322)/38.330810

In [ ]:
# -------------------------------------------------------------------------------------------------
# 단말이 측정범위를 벗어났는지 확인
# 2022.02.21 - 측정유형(userinfo2)이 행정동("행-")인 경우만 처리하는 것이 맞지 않나? 
#            - 행정동: 도로주행 측정, 테마: 놀이공원 등 걸어서 측정, 인빌딩: 건물내 걸어서 측정
#--------------------------------------------------------------------------------------------------
def out_measuring_range(mdata):
    ''' 단말이 측정범위를 벗어났는지 확인
        - 측정하는 행정동을 벗어나서 측정이 되는 경우
        - (아이디어) 행정동을 벗어남이 의심된다는 메시지 + 위치 지도 이미지도 함께 전송
        - return 'OUTRANGE'
    '''
    # 측정유형이 행정동인 경우에만 단말이 측정범위를 벗어났는지 확인한다.
    if not mdata.userInfo2.startswith("행-") : return None

    # 2.20 역지오코딩 - 도로명 주소로 반환되어 카카오지도 API로 대체
    # geolocator = Nominatim(user_agent="myGeolocator")
    # location = geolocator.reverse(str(mdata.latitude) + ',' + str(mdata.longitude))
    # Location(영서로, 학곡리, 춘천시, 강원도, 24408, 대한민국, (37.81069349300918, 127.7657987426381, 0.0))
    # print("out_measuring_range():", location)

    rest_api_key = "9daef46439c87ea1a53391feb26ebb8b"
    kakao = KakaoLocalAPI(rest_api_key)
    input_coord = "WGS84" # WGS84, WCONGNAMUL, CONGNAMUL, WTM, TM

    result = kakao.geo_coord2address(mdata.longitude, mdata.latitude, input_coord)
    # print("out_measuring_range():", result)
    # {'meta': {'total_count': 1}, 
    #   'documents': [
    #       {'road_address': None, 
    #       'address': 
    #       {'address_name': '강원 춘천시 동내면 사암리 산 121-1', 'region_1depth_name': '강원', 
    #       'region_2depth_name': '춘천시', 'region_3depth_name': '동내면 사암리', 'mountain_yn': 'Y', 
    #       'main_address_no': '121', 'sub_address_no': '1', 'zip_code': ''}}]}
    # meta,
    # document -> road_address
    #          -> address -> region_3depth_name
    # 좌표(위도,경도)로 찾은 주소와 어떤 것을 비교할지? 고민필요
    # userInfo1가 위도,경도 좌표로 변환한 행정동을 포함하고 있는지 확인
    try: 
        region_3depth_name = result['documents'][0]['address']['region_3depth_name'].split()[0]
        if mdata.userInfo1.find(region_3depth_name) == -1:
            return 'OUTRANGE'
        else:
            return None
    except Exception as e:
        print("out_measuring_range():", str(e))
        return None

In [ ]:
phone = Phone.object

In [9]:
rest_api_key = "9daef46439c87ea1a53391feb26ebb8b"
kakao = KakaoLocalAPI(rest_api_key)
input_coord = "WGS84" # WGS84, WCONGNAMUL, CONGNAMUL, WTM, TM

result = kakao.geo_coord2address(128.42987614, 34.84455475, input_coord)
result

{'meta': {'total_count': 1},
 'documents': [{'road_address': None,
   'address': {'address_name': '경남 통영시 정량동 1158-29',
    'region_1depth_name': '경남',
    'region_2depth_name': '통영시',
    'region_3depth_name': '정량동',
    'mountain_yn': 'N',
    'main_address_no': '1158',
    'sub_address_no': '29',
    'zip_code': ''}}]}